In [24]:
# Working on adding final tables and charts to tab1 dashboard

import pandas as pd
import numpy as np
import plotly.express as px

import tempfile
import sys
import re
import asyncio  # Import asyncio for Windows event loop policy

import panel as pn
import panel.widgets as pnw

# Adjust event loop policy for Windows
if sys.platform.startswith('win'):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

# Initialize Panel extension with required widgets
pn.config.theme = 'default'
pn.extension('plotly', 'tabulator', 'echarts')
pn.extension(design='material') #available design systems ['design', 'bootstrap', 'fast', 'material', 'native'].

import panel as pn

pn.extension(design='material')

# Define your custom CSS as a string
custom_css = """
/*Active tab background and text color */
.bk-tab.bk-active {
    background-color: #6200ea !important; /* Material Indigo */
    color: #ffffff !important;
}

/* Inactive tab background and text color */
.bk-tab {
    background-color: #e0e0e0 !important; /* Light Grey */
    color: #333333 !important;
}

/* Rounded corners to tabs */
.bk-tabs-header {
    border-radius: 8px;
    overflow: hidden;
}

/* Example: Customize hover effect on tabs */
.bk-tab:hover {
    background-color: #c5cae9 !important; /* Light Indigo */
    transition: background-color 0.3s ease;
}
.centered-text {
    text-align: center;
    font-weight: bold;
    color: #333333;
}

.description-text {
    text-align: center;
    color: #666666;
}
"""

# Append the custom CSS to Panel's raw CSS
pn.config.raw_css.append(custom_css)

# ---------------------------
# Data Loading and Preparation
# ---------------------------

def load_fmr_data(filepath):
    try:
        df_fmr = pd.read_csv(filepath)
        print("Original FMR DataFrame Columns:", df_fmr.columns.tolist())  # Debugging

        # Removing trailing spaces from column names
        df_fmr.columns = df_fmr.columns.str.strip()

        # Extracting first 5 digits and ensure it's 5 digits with leading zeros
        df_fmr['fips5'] = df_fmr['fips'].astype(str).str[:5].str.zfill(5)
        df_fl_fips = df_fmr[df_fmr['stusps'] == 'FL'].copy()
        print("Filtered FL FIPS DataFrame Columns:", df_fl_fips.columns.tolist())  # Debugging

        # Ensuring 'fips5' is included in the mask
        mask_fmr = ['COUNTY', 'stusps', 'pop2022', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4', 'fips5']
        df_fl_fmr = df_fl_fips[mask_fmr].rename(columns={
            'fmr_0': '0 Bedroom FMR ($)',
            'fmr_1': '1 Bedroom FMR ($)',
            'fmr_2': '2 Bedroom FMR ($)',
            'fmr_3': '3 Bedroom FMR ($)',
            'fmr_4': '4 Bedroom FMR ($)'
        }).copy()
        # print("Renamed FMR DataFrame Columns:", df_fl_fmr.columns.tolist())  # Debugging

        # Ensuring no leading/trailing spaces in 'COUNTY' column
        df_fl_fmr['COUNTY'] = df_fl_fmr['COUNTY'].str.strip()

        # Verify that '2 Bedroom FMR ($)' exists
        if '2 Bedroom FMR ($)' not in df_fl_fmr.columns:
            print("Error: '2 Bedroom FMR ($)' column not found after renaming.")
        else:
            print("'2 Bedroom FMR ($)' column successfully renamed and exists.")

        return df_fl_fips, df_fl_fmr
        
    except FileNotFoundError:
        pn.state.notifications.error("Error: 'FY25_FMRs.csv' not found.")
        sys.exit(1)
    except Exception as e:
        pn.state.notifications.error(f"Error loading FMR data: {e}")
        sys.exit(1)

def load_ami_data(filepath):
    try:
        df_ami = pd.read_csv(filepath)
        # print("Original AMI DataFrame Columns:", df_ami.columns.tolist())  # Debugging

        # Remove trailing spaces from column names
        df_ami.columns = df_ami.columns.str.strip()

        mask_ami_median = ['COUNTY', 'Median Income ($)']
        mask_ami_housing = ['COUNTY', 'AMI Category','1 Person Limit ($)','2 Person Limit ($)','3 Person Limit ($)','4 Person Limit ($)']
        mask_ami_rental = ['COUNTY', 'AMI Category','0 Bedroom Limit ($)','1 Bedroom Limit ($)','2 Bedroom Limit ($)','3 Bedroom Limit ($)','4 Bedroom Limit ($)']

        df_ami_median = df_ami[mask_ami_median].copy()
        df_ami_housing = df_ami[mask_ami_housing].copy()
        df_ami_rental = df_ami[mask_ami_rental].copy()

        # Optional: Ensure no leading/trailing spaces in 'COUNTY' column
        df_ami_median['COUNTY'] = df_ami_median['COUNTY'].str.strip()
        df_ami_housing['COUNTY'] = df_ami_housing['COUNTY'].str.strip()
        df_ami_rental['COUNTY'] = df_ami_rental['COUNTY'].str.strip()

        #print("AMI Median DataFrame Columns:", df_ami_median.columns.tolist())  # Debugging
        #print("AMI Housing DataFrame Columns:", df_ami_housing.columns.tolist())  # Debugging
        #print("AMI Rental DataFrame Columns:", df_ami_rental.columns.tolist())  # Debugging

        return df_ami_median, df_ami_housing, df_ami_rental
        
    except FileNotFoundError:
        pn.state.notifications.error("Error: 'fl_ami_income_limit.csv' not found.")
        sys.exit(1)
    except Exception as e:
        pn.state.notifications.error(f"Error loading AMI data: {e}")
        sys.exit(1)

def load_jobs_data(filepath):
    try:
        df_jobs = pd.read_csv(filepath)
        #print("Original Jobs DataFrame Columns:", df_jobs.columns.tolist())  # Debugging

        # Remove trailing spaces from column names
        df_jobs.columns = df_jobs.columns.str.strip()

        mask_ami_jobs = [
            'COUNTY', 'Geography', 'Occupation', 'Median Hourly Wage (2023 $)', 
            'Maximum Affordable Rent (30% of Income)', '2023 HUD 2BR Fair Market Rent',
            '% Income Needed for 2 BR, Median Wage Worker', '# of Workers in 2022',
            'Annual Wage as a % AMI for a Family of 3'
        ]
        df_ami_jobs = df_jobs[mask_ami_jobs].copy()

        # Clean up 'COUNTY' column
        df_ami_jobs['COUNTY'] = df_ami_jobs['COUNTY'].str.strip()

        # Split 'COUNTY' into a list of counties
        df_ami_jobs['COUNTY_LIST'] = df_ami_jobs['COUNTY'].str.split(',')

        # Remove leading/trailing spaces from each county in the list
        df_ami_jobs['COUNTY_LIST'] = df_ami_jobs['COUNTY_LIST'].apply(lambda x: [c.strip() for c in x])

        # Explode the DataFrame so each county has its own row
        df_ami_jobs = df_ami_jobs.explode('COUNTY_LIST')

        # Now, 'COUNTY_LIST' contains single county names
        # Rename 'COUNTY_LIST' back to 'COUNTY' for consistency
        df_ami_jobs['COUNTY'] = df_ami_jobs['COUNTY_LIST']
        df_ami_jobs.drop(columns=['COUNTY_LIST'], inplace=True)

       # print("Filtered Jobs DataFrame Columns:", df_ami_jobs.columns.tolist())  # Debugging
       # print("Sample Jobs DataFrame after exploding 'COUNTY':")
       # print(df_ami_jobs.head())  # Debugging

        return df_ami_jobs
    except FileNotFoundError:
        pn.state.notifications.error("Error: 'fl_ami_occupation_wage.csv' not found.")
        sys.exit(1)
    except Exception as e:
        pn.state.notifications.error(f"Error loading Jobs data: {e}")
        sys.exit(1)



# New function to load Housing Cost Burden data


def load_burden_data(filepath):
    try:
        df_burden = pd.read_csv(filepath)

        # Remove trailing spaces from column names
        df_burden.columns = df_burden.columns.str.strip()

        # Table masks for Everyone, Renters, and Homeowners
        mask_burden_all = ['Geography', 'Household Income', 'ALL: 30% or less', 'ALL: 30.1-50%', 'ALL: More than 50%']
        mask_burden_renters = ['Geography', 'Household Income', 'RENTERS: 30% or less', 'RENTERS: 30.1-50%', 'RENTERS: More than 50%']
        mask_burden_homeowners = ['Geography', 'Household Income', 'HOMEOWNERS: 30% or less', 'HOMEOWNERS: 30.1-50%', 'HOMEOWNERS: More than 50%']

        # Dataframes with a little clean-up
        df_fl_burden_all = df_burden[mask_burden_all].rename(columns={
            'ALL: 30% or less': '30% or less', 
            'ALL: 30.1-50%': '30.1-50%', 
            'ALL: More than 50%': 'More than 50%'
        }).copy()

        df_fl_burden_renters = df_burden[mask_burden_renters].rename(columns={
            'RENTERS: 30% or less': '30% or less', 
            'RENTERS: 30.1-50%': '30.1-50%', 
            'RENTERS: More than 50%': 'More than 50%'
        }).copy()

        df_fl_burden_homeowners = df_burden[mask_burden_homeowners].rename(columns={
            'HOMEOWNERS: 30% or less': '30% or less', 
            'HOMEOWNERS: 30.1-50%': '30.1-50%', 
            'HOMEOWNERS: More than 50%': 'More than 50%'
        }).copy()

        # Convert columns to numeric if possible
        numeric_cols = ['30% or less', '30.1-50%', 'More than 50%']
        df_fl_burden_all[numeric_cols] = df_fl_burden_all[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
        df_fl_burden_renters[numeric_cols] = df_fl_burden_renters[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
        df_fl_burden_homeowners[numeric_cols] = df_fl_burden_homeowners[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

        # Rename 'Geography' to 'COUNTY' to match existing datasets
        df_fl_burden_all.rename(columns={'Geography': 'COUNTY'}, inplace=True)
        df_fl_burden_renters.rename(columns={'Geography': 'COUNTY'}, inplace=True)
        df_fl_burden_homeowners.rename(columns={'Geography': 'COUNTY'}, inplace=True)

        return df_fl_burden_all, df_fl_burden_renters, df_fl_burden_homeowners
    except FileNotFoundError:
        pn.state.notifications.error("Error: 'fl_ami_cost_burden_all.csv' not found.")
        sys.exit(1)
    except Exception as e:
        pn.state.notifications.error(f"Error loading Burden data: {e}")
        sys.exit(1)

def calculate_totals(df):
    numeric_cols = ['30% or less', '30.1-50%', 'More than 50%']

    # Calculate the totals row for numeric columns and ensure they are integers
    totals_row = df[numeric_cols].sum().astype(int)
    totals_row['COUNTY'] = 'TOTALS'
    totals_row['Household Income'] = ''

    # Calculate the 'Number' column for totals: sum of '30.1-50%' and 'More than 50%'
    number_total = df['30.1-50%'].sum() + df['More than 50%'].sum()
    totals_row['Number'] = int(number_total)  # Ensure 'Number' is integer

    # Calculate the 'Percentage' column for totals
    total_sum = df['30% or less'].sum() + df['30.1-50%'].sum() + df['More than 50%'].sum()
    if total_sum != 0:
        percentage_total = (number_total / total_sum) * 100
    else:
        percentage_total = 0.0
    totals_row['Percentage'] = float(percentage_total)

    # Concatenate the totals row to the original DataFrame
    df_totals = pd.concat([df, pd.DataFrame([totals_row])], ignore_index=True)

    # Adding the 'TOTALS' column: sum of numeric columns for each row
    df_totals['TOTALS'] = df_totals[numeric_cols].sum(axis=1).astype(int)

    # Ensure 'Number' and 'Percentage' columns have correct data types in the DataFrame
    df_totals['Number'] = df_totals['Number'].astype(int)
    df_totals['Percentage'] = df_totals['Percentage'].astype(float)

    # Define the expected columns order
    expected_columns = [
        'COUNTY',
        'Household Income',
        '30% or less',
        '30.1-50%',
        'More than 50%',
        'TOTALS',
        'Number',
        'Percentage'
    ]

    # Reorder the DataFrame columns
    df_totals = df_totals[expected_columns]

    return df_totals

def create_multiindex_columns(df):
    # Ensure that DF has the expected number of columns
    assert len(df.columns) == 8, f'Expected 8 columns, but got {len(df.columns)} columns.'

    # Define the new multi-level column structure, including the new columns
    new_columns = [
        ('', 'COUNTY'),
        ('', 'Household Income'),
        ('Housing Cost Burden Range - # of Households', '30% or less'),
        ('Housing Cost Burden Range - # of Households', '30.1-50%'),
        ('Housing Cost Burden Range - # of Households', 'More than 50%'),
        ('', 'TOTALS'),
        ('Housing Cost Burden > 30%', 'Number'),
        ('Housing Cost Burden > 30%', 'Percentage')
    ]
    df.columns = pd.MultiIndex.from_tuples(new_columns)
    return df

def add_columns(df):
    # Ensure relevant columns are numeric for calculation
    numeric_cols = ['30% or less', '30.1-50%', 'More than 50%']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    # Add the 'Number' column: sum of '30.1-50%' and 'More than 50%'
    df['Number'] = df['30.1-50%'] + df['More than 50%']
    df['Number'] = df['Number'].astype(int)  # Ensure 'Number' is integer

    # Add the 'Percentage' column
    total_sum = df['30% or less'] + df['30.1-50%'] + df['More than 50%']
    df['Percentage'] = (df['30.1-50%'] + df['More than 50%']) / total_sum

    # Replace any potential NaNs (from division by zero) with 0
    df['Percentage'] = df['Percentage'].replace([np.inf, -np.inf], np.nan).fillna(0)
    df['Percentage'] = df['Percentage'].astype(float)  # Ensure 'Percentage' is float
    df['Percentage'] = df['Percentage'] * 100  # Formatting to create xx.x%

    return df

def load_housing_data(filepath):
    df_housing = pd.read_csv(filepath)

    # Remove trailing spaces from column names
    df_housing.columns = df_housing.columns.str.strip()

    # Define the required columns
    mask_avail_housing = [
        'COUNTY',
        'Total Housing Units',
        'Occupied Housing Units',
        'Vacant Hosing Units',
        'Vacancy Rate: Homeowner',
        'Vacancy Rate: Rental'
    ]

    # Select and rename necessary columns
    df_avail_housing = df_housing[mask_avail_housing].rename(columns={ 
        'Total Housing Units': 'Available Housing Units',
        'Occupied Housing Units': 'Occupied Snapshot',
        'Vacant Hosing Units': 'Vacancy Snapshot'
    }).copy()

    # Standardize 'COUNTY' names to lowercase for consistency
    df_avail_housing['COUNTY'] = df_avail_housing['COUNTY'].str.lower()

    return df_avail_housing

def load_pop_data(filepath):
    df_pop = pd.read_csv(filepath)

    # Remove trailing spaces from column names
    df_pop.columns = df_pop.columns.str.strip()
   
    # Rename columns for consistency
    df_pop = df_pop.rename(columns={ 
        'pop2022': 'Population: 2022'
    }).copy()

    # Standardize 'COUNTY' names to lowercase for consistency
    df_pop['COUNTY'] = df_pop['COUNTY'].str.lower()

    return df_pop


# Load datasets
df_fl_fips, df_fl_fmr = load_fmr_data('FY25_FMRs.csv')
df_ami_median, df_ami_housing, df_ami_rental = load_ami_data('fl_ami_income_limit.csv')
df_ami_jobs = load_jobs_data('fl_ami_occupation_wage.csv')
df_pop = load_pop_data('fl_pop.csv')
df_avail_housing = load_housing_data('fl_housing.csv')

# Load Housing Cost Burden data
df_fl_burden_all, df_fl_burden_renters, df_fl_burden_homeowners = load_burden_data('fl_ami_cost_burden_all.csv')

# Load and preprocess tab1 barchart data
df_census_structure = pd.read_csv('fl_census_structure.csv')
df_census_built = pd.read_csv('fl_census_built.csv')
df_census_rooms = pd.read_csv('fl_census_rooms.csv')
df_census_bdrms = pd.read_csv('fl_census_bdrm.csv')
df_census_value = pd.read_csv('fl_census_value.csv')


#drop unused row
df_census_structure.drop(columns=['Unnamed: 0'], inplace=True)
df_census_built.drop(columns=['Unnamed: 0'], inplace=True)
df_census_rooms.drop(columns=['Unnamed: 0'], inplace=True)
df_census_bdrms.drop(columns=['Unnamed: 0'], inplace=True)
df_census_value.drop(columns=['Unnamed: 0'], inplace=True)

# Apply 'add_columns' to each DataFrame
df_fl_burden_all = add_columns(df_fl_burden_all)
df_fl_burden_renters = add_columns(df_fl_burden_renters)
df_fl_burden_homeowners = add_columns(df_fl_burden_homeowners)

# Define a custom color palette for each Unit Type
custom_colors = ['#E74C3C', '#3498DB', '#9B59B6', '#1ABC9C', '#F39C12', '#D35400', '#8E44AD', '#2ECC71', '#E67E22', '#16A085']

# ---------------------------
# Normalize County Names
# ---------------------------

def normalize_county_name(name):
    name = name.lower()
    name = name.replace('county', '')
    name = name.replace('-', ' ')
    name = name.replace('.', '')
    name = name.strip()
    name = re.sub(r'\bst\b', 'saint', name)  # Replace 'st' with 'saint'
    name = re.sub(r'\s+', ' ', name)  # Replace multiple spaces with one
    return name

# Normalize 'COUNTY' column in df_fl_burden_* DataFrames
for df in [df_fl_burden_all, df_fl_burden_renters, df_fl_burden_homeowners]:
    df['COUNTY_NORM'] = df['COUNTY'].apply(normalize_county_name)

# Normalize 'COUNTY' column in other DataFrames (if needed)
df_fl_fmr['COUNTY_NORM'] = df_fl_fmr['COUNTY'].apply(normalize_county_name)
df_ami_median['COUNTY_NORM'] = df_ami_median['COUNTY'].apply(normalize_county_name)
df_ami_housing['COUNTY_NORM'] = df_ami_housing['COUNTY'].apply(normalize_county_name)
df_ami_rental['COUNTY_NORM'] = df_ami_rental['COUNTY'].apply(normalize_county_name)
df_ami_jobs['COUNTY_NORM'] = df_ami_jobs['COUNTY'].apply(normalize_county_name)
df_pop['COUNTY_NORM'] = df_pop['COUNTY'].apply(normalize_county_name)
df_avail_housing['COUNTY_NORM'] = df_avail_housing['COUNTY'].apply(normalize_county_name)
#df_census_structure['COUNTY_NORM'] = df_census_structure['COUNTY'].apply(normalize_county_name)
#df_census_built['COUNTY_NORM'] = df_census_built['COUNTY'].apply(normalize_county_name)
#df_census_rooms['COUNTY_NORM'] = df_census_rooms['COUNTY'].apply(normalize_county_name)
#df_census_bdrms['COUNTY_NORM'] = df_census_bdrms['COUNTY'].apply(normalize_county_name)
#df_census_value['COUNTY_NORM'] = df_census_value['COUNTY'].apply(normalize_county_name)

# Update the list of county names for the widget
county_names = sorted(df_fl_fmr['COUNTY'].unique().tolist())

# ---------------------------
# Widget Definition
# ---------------------------

select_county = pn.widgets.Select(
    name='Select County',
    options=['All Counties'] + county_names,
    value='All Counties',
    sizing_mode='stretch_width'
)

# Loading spinner (optional, since updates are reactive and likely fast)
loading_spinner = pn.indicators.LoadingSpinner(value=False, width=50, height=50, visible=False)

# ---------------------------
# Create Reactive Tables for New Pane
# ---------------------------

# Bar Chart function
# Table function
def create_plot_table(county, table_type='structure'):
    df_structure = df_census_structure[df_census_structure['COUNTY'] == county]
    df_built = df_census_built[df_census_built['COUNTY'] == county]
    df_bedrooms = df_census_bdrms[df_census_bdrms['COUNTY'] == county]
    df_rooms = df_census_rooms[df_census_rooms['COUNTY'] == county]
    df_value = df_census_value[df_census_value['COUNTY'] == county]
    #return removing due to early exit issue

    # Ensure there is data for the selected county
    if df_structure.empty:
        return pd.DataFrame([{"Message": "No data available for the selected county"}])
        
    # Select the table based on table_type
    # Barplot and table: Structure
    if table_type == 'structure':
        table = df_structure.drop(columns=['COUNTY']).transpose().rename(columns={df_structure.index[0]: county})
        table.reset_index(inplace=False)
    # Rename columns based on the number of columns
        if table.shape[1] == 2:
            table.columns = ['Unit Type', 'Value']
        else:
            table.columns = ['Unit Type'] + [f"Value {i}" for i in range(1, table.shape[1])]

    # Barplot and table: Year Built
    elif table_type == 'built':
        table = df_built.drop(columns=['COUNTY']).transpose().rename(columns={df_built.index[0]: county})
        table.reset_index(inplace=False)
    # Rename columns based on the number of columns
        if table.shape[1] == 2:
            table.columns = ['Year Built', 'Value']
        else:
            table.columns = ['Year Built'] + [f"Value {i}" for i in range(1, table.shape[1])]
    
    # Barblot and table: Bedroom Count
    elif table_type == 'bedrooms':
        table = df_bedrooms.drop(columns=['COUNTY']).transpose().rename(columns={df_bedrooms.index[0]: county})
        table.reset_index(inplace=False)
    # Rename columns based on the number of columns
        if table.shape[1] == 2:
            table.columns = ['Bedroom Count', 'Value']
        else:
            table.columns = ['Bedroom Count'] + [f"Value {i}" for i in range(1, table.shape[1])]
    
    # Barblot and table: Room Count
    elif table_type == 'rooms':
        table = df_rooms.drop(columns=['COUNTY']).transpose().rename(columns={df_rooms.index[0]: county})
        table.reset_index(inplace=False)
    # Rename columns based on the number of columns
        if table.shape[1] == 2:
            table.columns = ['Bedroom Count', 'Value']
        else:
            table.columns = ['Bedroom Count'] + [f"Value {i}" for i in range(1, table.shape[1])]
    
    # Barblot and table: Room Count
    elif table_type == 'value':
        table = df_value.drop(columns=['COUNTY']).transpose().rename(columns={df_value.index[0]: county})
        table.reset_index(inplace=False)
    # Rename columns based on the number of columns
        if table.shape[1] == 2:
            table.columns = ['Home Count', 'Value']
        else:
            table.columns = ['Home Count'] + [f"Value {i}" for i in range(1, table.shape[1])]
            
    else:
        raise ValueError("Invalid table_type specified. Choose 'rooms', 'bdrms', 'structure' or 'built'.")
    
    return table


def plot_chart_structure(county):
    df = df_census_structure[df_census_structure['COUNTY'] == county]
    df_melted = df.melt(id_vars='COUNTY', var_name='Unit Type', value_name='Value')
    fig_unit_type = px.bar(df_melted, x='Unit Type', y='Value', title=f"Housing Units by Type for {county}", color='Unit Type', color_discrete_sequence=custom_colors)
    fig_unit_type.update_layout(xaxis_title='Unit Type', yaxis_title='Count of Units')
    return fig_unit_type

def plot_chart_built(county):
    df = df_census_built[df_census_built['COUNTY'] == county]
    df_melted = df.melt(id_vars='COUNTY', var_name='Year Built', value_name='Value')
    fig_year_built = px.bar(df_melted, x='Year Built', y='Value', title=f"Housing Units by Year Built for {county}", color='Year Built', color_discrete_sequence=custom_colors)
    fig_year_built.update_layout(xaxis_title='Year Built', yaxis_title='Count of Units')
    return fig_year_built

def plot_chart_bedrooms(county):
    df = df_census_bdrms[df_census_bdrms['COUNTY'] == county]
    df_melted = df.melt(id_vars='COUNTY', var_name='Bedroom Count', value_name='Value')
    fig_bedroom_count = px.bar(df_melted, x='Bedroom Count', y='Value', title=f"Housing Units by Bedroom Count for {county}", color='Bedroom Count', color_discrete_sequence=custom_colors)
    fig_bedroom_count.update_layout(xaxis_title='Bedroom Count', yaxis_title='Count of Units')
    return fig_bedroom_count

def plot_chart_rooms(county):
    df = df_census_rooms[df_census_rooms['COUNTY'] == county]
    df_melted = df.melt(id_vars='COUNTY', var_name='Room Count', value_name='Value')
    fig_room_count = px.bar(df_melted, x='Room Count', y='Value', title=f"Housing Units by Occupancy Status for {county}", color='Room Count', color_discrete_sequence=custom_colors)
    fig_room_count.update_layout(xaxis_title='Room Count', yaxis_title='Count of Units')
    return fig_room_count

def plot_chart_value(county):
    df = df_census_value[df_census_value['COUNTY'] == county]
    df_melted = df.melt(id_vars='COUNTY', var_name='Home Value', value_name='Value')
    fig_value = px.bar(df_melted, x='Home Value', y='Value', title=f"Housing Units by Value for {county}", color='Home Value', color_discrete_sequence=custom_colors)
    fig_value.update_layout(xaxis_title='Home Value', yaxis_title='USD')
    return fig_value

# Function to create tables for each category
def create_burden_table(df_burden, county):
    # Normalize the selected county name
    county_norm = normalize_county_name(county)
    print(f"Normalized selected county: '{county}' -> '{county_norm}'")  # Debugging
    
    filtered_df = df_burden[df_burden['COUNTY_NORM'] == county_norm]
    if filtered_df.empty:
        print(f"No data found for county: {county_norm}")  # Debugging
        return pn.pane.Markdown("No data available for this county.", sizing_mode='stretch_width')
    df_with_totals = calculate_totals(filtered_df)
    df_with_multi = create_multiindex_columns(df_with_totals)

    # Use Styler to create multi-level headers and apply custom CSS
    styler = df_with_multi.style

    # Hiding Index
    styler.hide(axis='index')

    # Set table styles
    styler.set_table_styles([
        # Style for the multi-level header
        {
            'selector': 'table',
            'props': [
                ('border-collapse', 'collapse'),
                ('border-spacing', '0px'),
                ('font-size', '1em'),
                ('font-family', 'Verdana, sans-serif'),
            ]
        },
        { 
            'selector': 'th',
            'props': [
                ('border', '1px solid black'), 
                ('padding', '5px'), 
                ('text-align', 'center'),
                ('background-color', '#f0f0f0'),
                ('border-bottom', '1px solid #ccc'),
            ]
        },
        {
            'selector': 'td',
            'props': [
                ('border', '1px solid black'), 
                ('padding', '5px')
            ]
        },
        # Hover effect on table rows
        {
            'selector': 'tr:hover td',
            'props': [
                ('background-color', '#f1f1f1'),    # Light hover color
            ]
        },
    ])

    # Highlight specific columns
    styler.applymap(lambda x: 'background-color: #90ee90', subset=pd.IndexSlice[:, ('Housing Cost Burden Range - # of Households', '30% or less')])
    styler.applymap(lambda x: 'background-color: #ffff99', subset=pd.IndexSlice[:, ('Housing Cost Burden Range - # of Households', '30.1-50%')])
    styler.applymap(lambda x: 'background-color: #ffcccb', subset=pd.IndexSlice[:, ('Housing Cost Burden Range - # of Households', 'More than 50%')])
    styler.applymap(lambda x: 'background-color: #ffd700', subset=pd.IndexSlice[:, ('Housing Cost Burden > 30%', 'Number')])
    styler.applymap(lambda x: 'background-color: #ffd700', subset=pd.IndexSlice[:, ('Housing Cost Burden > 30%', 'Percentage')])

    # Format percentage
    styler.format(
        {('Housing Cost Burden > 30%', 'Percentage'): '{:.2f}%',
         ('Housing Cost Burden > 30%', 'Number'): '{:,}'  # Format 'Number' with thousand separators
        }
    )

    # Render the styled table to HTML
    df_html = styler.to_html()

    return pn.pane.HTML(df_html, sizing_mode='stretch_width')

# Functions for each table, reactive to 'select_county'
@pn.depends(select_county.param.value)
def table_all(county):
    if county == 'All Counties':
        return create_burden_table(df_fl_burden_all, 'All Counties')
    else:
        return create_burden_table(df_fl_burden_all, county)

@pn.depends(select_county.param.value)
def table_renters(county):
    if county == 'All Counties':
        return create_burden_table(df_fl_burden_renters, 'All Counties')
    else:
        return create_burden_table(df_fl_burden_renters, county)

@pn.depends(select_county.param.value)
def table_homeowners(county):
    if county == 'All Counties':
        return create_burden_table(df_fl_burden_homeowners, 'All Counties')
    else:
        return create_burden_table(df_fl_burden_homeowners, county)

# Create a layout for the new pane, reactive to 'select_county'
@pn.depends(select_county.param.value)
def burden_layout(county):
    if county == 'All Counties':
        return pn.Column(
            pn.pane.Markdown("## Housing Cost Burden"),
            pn.pane.Markdown("Select a County from the list to see the housing cost burden data."),
            sizing_mode='stretch_width'
        )
    else:
        return pn.Column(
            pn.pane.Markdown("## Housing Cost Burden"),
            pn.layout.Divider(),
            pn.pane.Markdown("### All Households"),
            table_all,
            pn.layout.Divider(),
            pn.pane.Markdown("### Renters"),
            table_renters,
            pn.layout.Divider(),
            pn.pane.Markdown("### Homeowners"),
            table_homeowners,
            sizing_mode='stretch_width'
        )

# ---------------------------
# Create Barplots and Tables
# ---------------------------

# Dynamic update function for both plot and table
@pn.depends(select_county.param.value)
def update_plot_structure(county):
    # Update title
    #title_pane.object = f"## Housing Unit Structure for {county}"
    
    # Update plot and table
    return pn.Column(
        plot_chart_structure(county),
        pn.pane.Markdown("### Structure Table View", css_classes=['centered-text']),
        pn.widgets.DataFrame(create_plot_table(county, table_type='structure'), height=200, width=600),
        sizing_mode='stretch_width'
    )

@pn.depends(select_county.param.value)
def update_plot_built(county):
    # Update title
    #title_pane.object = f"## Housing Unit Structure for {county}"
    
    # Update plot and table
    return pn.Column(
        plot_chart_built(county),
        pn.pane.Markdown("### Year Built Table View", css_classes=['centered-text']),
        pn.widgets.DataFrame(create_plot_table(county, table_type='built'), height=200, width=600),
        sizing_mode='stretch_width'
    )

@pn.depends(select_county.param.value)
def update_plot_bedrooms(county):
    # Update title
    #title_pane.object = f"## Housing Unit Structure for {county}"
    
    # Update plot and table
    return pn.Column(
        plot_chart_bedrooms(county),
        pn.pane.Markdown("### No. Bedrooms Table View", css_classes=['centered-text']),
        pn.widgets.DataFrame(create_plot_table(county, table_type='bedrooms'), height=200, width=600),
        sizing_mode='stretch_width'
    )

@pn.depends(select_county.param.value)
def update_plot_rooms(county):
    # Update title
    #title_pane.object = f"## Housing Unit Structure for {county}"
    
    # Update plot and table
    return pn.Column(
        plot_chart_rooms(county),
        pn.pane.Markdown("### No. Rooms Table View", css_classes=['centered-text']),
        pn.widgets.DataFrame(create_plot_table(county, table_type='rooms'), height=200, width=600),
        sizing_mode='stretch_width'
    )

@pn.depends(select_county.param.value)
def update_plot_value(county):
    # Update title
    #title_pane.object = f"## Housing Unit Structure for {county}"
    
    # Update plot and table
    return pn.Column(
        plot_chart_value(county),
        pn.pane.Markdown("### Home Value Table View", css_classes=['centered-text']),
        pn.widgets.DataFrame(create_plot_table(county, table_type='value'), height=200, width=600),
        sizing_mode='stretch_width'
    )

# ---------------------------
# Create Tabulator Tables
# ---------------------------

# Function to create table layouts with download buttons
def create_table_layout(report_name, download_button, datasource_url, tabulator):
    return pn.Column(
        pn.pane.Markdown(f"## {report_name}"),
        pn.Row(download_button, pn.pane.Markdown(f"[DataSource]({datasource_url})")),
        tabulator,
        sizing_mode='stretch_width'
    )

# Create Tabulator tables for each dataset
tabulator_1 = pn.widgets.Tabulator(
    df_fl_fmr, 
    height=200,  # Increased height to display more rows
    sizing_mode='stretch_width', 
    name="FMR Data",
    visible=False,  # Initially hidden
    pagination='local',  # Enable local pagination
    page_size=50,  # Set 50 rows per page
    selectable='checkbox',  # Optional: Enable row selection
    sortable=True           # Optional: Enable column sorting
)

tabulator_2 = pn.widgets.Tabulator(
    df_ami_median, 
    height=150, 
    sizing_mode='stretch_width', 
    name="AMI Median Income",
    visible=False,  
    pagination='local',
    page_size=50,
    selectable='checkbox',
    sortable=True
)

tabulator_3 = pn.widgets.Tabulator(
    df_ami_housing, 
    height=400, 
    sizing_mode='stretch_width', 
    name="AMI Housing Limits",
    visible=False,  
    pagination='local',
    page_size=50,
    selectable='checkbox',
    sortable=True
)

tabulator_4 = pn.widgets.Tabulator(
    df_ami_rental, 
    height=400, 
    sizing_mode='stretch_width', 
    name="AMI Rental Limits",
    visible=False,  
    pagination='local',
    page_size=50,
    selectable='checkbox',
    sortable=True
)

tabulator_5 = pn.widgets.Tabulator(
    df_ami_jobs, 
    height=600, 
    sizing_mode='stretch_width', 
    name="Wage and Occupation",
    visible=False,  
    pagination='local',
    page_size=50,
    selectable='checkbox',
    sortable=True
)

# Create download buttons for all tables
def create_csv_button(df, filename):
    def csv_callback():
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
        df.to_csv(temp_file.name, index=False)
        return temp_file.name
    return pn.widgets.FileDownload(callback=csv_callback, filename=filename, button_type='primary')

download_button_1 = create_csv_button(df_fl_fmr, "fmr.csv")
download_button_2 = create_csv_button(df_ami_median, "ami_median.csv")
download_button_3 = create_csv_button(df_ami_housing, "ami_housing.csv")
download_button_4 = create_csv_button(df_ami_rental, "ami_rental.csv")
download_button_5 = create_csv_button(df_ami_jobs, "ami_jobs.csv")

# ---------------------------
# Define Data Sources and Reports
# ---------------------------

# FMR Data
datasource_fmr = 'U.S. Department of Housing and Urban Development (HUD)'
report_name_fmr = 'Fair Market Rents 2025 County Level Data'
datasource_url_fmr = 'https://www.huduser.gov/portal/datasets/fmr.html#data_2025'

# AMI Income Limits Data
datasource_ami = 'Florida Housing Finance Corporation'
report_name_ami = 'SHIP and HHRP - 2024 Combined Income and Rent Limits (Eff. 4-1-2024)'
datasource_url_ami = 'https://www.floridahousing.org/owners-and-managers/compliance/income-limits'

# Wage and Occupation Data
datasource_occupation = 'Florida Housing Data Clearinghouse'
report_name_occupation = 'Wage and Rent Comparison by Occupation, 2023'
datasource_url_occupation = 'http://flhousingdata.shimberg.ufl.edu/affordability/results?nid=100000'

# ---------------------------
# Widget Definition
# ---------------------------
# Create individual table layouts
table_1_layout = create_table_layout(report_name_fmr, download_button_1, datasource_url_fmr, tabulator_1)
table_2_layout = create_table_layout(report_name_ami, download_button_2, datasource_url_ami, tabulator_2)
table_3_layout = pn.Column(
    pn.pane.Markdown(f"## Housing Limits by AMI"),
    pn.Row(download_button_3),
    tabulator_3,
    sizing_mode='stretch_width'
)
table_4_layout = pn.Column(
    pn.pane.Markdown("## Rental Limits by AMI"),
    pn.Row(download_button_4),
    tabulator_4,
    sizing_mode='stretch_width'
)
table_5_layout = create_table_layout(report_name_occupation, download_button_5, datasource_url_occupation, tabulator_5)

# Function to create styled number indicators using Markdown panes
def create_number_indicator(title, value, font_size='24px', color='#2a9d8f'):
    """
    Creates a styled Markdown pane for number indicators.

    Parameters:
    - title (str): The title of the indicator.
    - value (int/float/str): The value to display.
    - font_size (str): Font size (e.g., '24px').
    - color (str): Font color (e.g., '#2a9d8f').

    Returns:
    - pn.pane.Markdown: A styled Markdown pane.
    """
    return pn.pane.Markdown(
        f"### {title}\n\n<span style='font-size:{font_size}; color:{color};'>{value}</span>",
        width=200,
        height=100
    )

# ---------------------------
# Initialize Indicators and Gauges with 'all' Row's Values
# ---------------------------

def get_all_values(df_pop, df_avail_housing):
    # Extract 'all' row data from df_pop
    pop_all_row = df_pop[df_pop['COUNTY_NORM'] == 'all counties']
    pop_value = pop_all_row['Population: 2022'].values[0] if not pop_all_row.empty else "N/A"
    
    # Extract 'all' row data from df_avail_housing
    avail_housing_all_row = df_avail_housing[df_avail_housing['COUNTY_NORM'] == 'all counties']
    if not avail_housing_all_row.empty:
        avail_housing = avail_housing_all_row['Available Housing Units'].values[0]
        occupied = avail_housing_all_row['Occupied Snapshot'].values[0]
        vacancy = avail_housing_all_row['Vacancy Snapshot'].values[0]
        homeowner_vacancy = avail_housing_all_row['Vacancy Rate: Homeowner'].values[0]
        rental_vacancy = avail_housing_all_row['Vacancy Rate: Rental'].values[0]
    else:
        avail_housing = "N/A"
        occupied = "N/A"
        vacancy = "N/A"
        homeowner_vacancy = 0
        rental_vacancy = 0
    
    return pop_value, avail_housing, occupied, vacancy, homeowner_vacancy, rental_vacancy

# Get 'all' values
pop_value, avail_housing, occupied, vacancy, homeowner_vacancy, rental_vacancy = get_all_values(df_pop, df_avail_housing)

print(f' homewner_vacancy is, {homeowner_vacancy}') # debugging
print(f' homewner_vacancy type, {type(homeowner_vacancy)}') # debugging

# Initialize number indicators with 'all' row's values
pop_indicator = create_number_indicator('Population: 2022', f"{int(pop_value):,}", font_size='24px', color='#2a9d8f')
available_housing_indicator = create_number_indicator('Available Housing Units', f"{int(avail_housing):,}", font_size='24px', color='#2a9d8f')
occupied_indicator = create_number_indicator('Occupied Snapshot', f"{int(occupied):,}", font_size='24px', color='#2a9d8f')
vacancy_indicator = create_number_indicator('Vacancy Snapshot', f"{int(vacancy):,}", font_size='24px', color='#2a9d8f')


# Define gauge indicators
homeowner_vacancy_gauge = pn.indicators.Gauge(
    name='Homeowner',
    value=0,
    bounds=(0, 4),
    width=200,
    colors=[(0.25, 'red'), (0.5, 'gold'), (1, 'green')],
    height=200,
)

rental_vacancy_gauge = pn.indicators.Gauge(
    name='Renters',
    value=0,
    bounds=(0, 50),
    width=200,
    colors=[(0.025, 'red'), (0.035, 'gold'), (1, 'green')],
    height=200,
)

# Initialize gauge indicators with 'all' row's values
if isinstance(homeowner_vacancy, (int, float)):
    homeowner_vacancy_gauge.value = homeowner_vacancy
else:
    homeowner_vacancy_gauge.value = 0  # Default to 0 if invalid

if isinstance(rental_vacancy, (int, float)):
    rental_vacancy_gauge.value = rental_vacancy
else:
    rental_vacancy_gauge.value = 0  # Default to 0 if invalid

# ---------------------------
# Choropleth Map Function (existing)
# ---------------------------

def create_map(selected_county):
    """
    Creates a Choropleth map based on the selected county.
    """
    if selected_county == 'All Counties':
        df = df_fl_fmr.copy()
        title = 'Florida Counties Map - Population 2022'
    else:
        df = df_fl_fmr[df_fl_fmr['COUNTY'] == selected_county].copy()
        title = f'Florida County Map - {selected_county} (Population 2022)'

    # Create the Choropleth map
    fig = px.choropleth(
        df,
        geojson='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json',
        locations='fips5',
        color='pop2022',  # Use numerical column for color scaling
        scope='usa',
        labels={'COUNTY': 'FL Counties'},
        title=title,
        hover_data=['COUNTY', 'pop2022'],  # Ensure these columns exist
        template='plotly_dark'
    )

    # Update the map's geographical properties
    fig.update_geos(fitbounds="locations", visible=False)

    return fig

# ---------------------------
# Panel Widgets and Plotly Pane
# ---------------------------

# Plotly Pane to display the Choropleth map
initial_fig = create_map('All Counties')
plotly_pane = pn.pane.Plotly(initial_fig, sizing_mode='stretch_width', height=400)

# Create a Plotly pane to hold the plot generated by plot_chart_structure
structure_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width')
bedrooms_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width')
rooms_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width')
value_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width')
built_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width')

# ---------------------------
# Reactive Update Function (corrected)
# ---------------------------

def update_dashboard(event):
    selected_county = event.new.strip().lower()  # Remove any leading/trailing spaces and lowercase
    loading_spinner.visible = True  # Show loading spinner

    # Normalize the selected county name
    county_norm = normalize_county_name(event.new)
    print(f"Normalized selected county: '{event.new}' -> '{county_norm}'")  # Debugging

    # Filter data based on selected county
    if event.new == 'All Counties':
        # Access the 'all' row by filtering
        pop_filtered = df_pop[df_pop['COUNTY_NORM'] == 'all counties']# changed COUNTRY_NORM
        avail_housing_filtered = df_avail_housing[df_avail_housing['COUNTY_NORM'] == 'all counties']# changed COUNTRY_NORM

        if not pop_filtered.empty:
            pop_value = pop_filtered['Population: 2022'].values[0]
        else:
            print("Error: 'all' row not found in df_pop.")
            pop_value = "N/A"

        if not avail_housing_filtered.empty:
            avail_housing = avail_housing_filtered['Available Housing Units'].values[0]
            occupied = avail_housing_filtered['Occupied Snapshot'].values[0]
            vacancy = avail_housing_filtered['Vacancy Snapshot'].values[0]
            homeowner_vacancy = avail_housing_filtered['Vacancy Rate: Homeowner'].values[0]
            rental_vacancy = avail_housing_filtered['Vacancy Rate: Rental'].values[0]
        else:
            print("Error: 'all' row not found in df_avail_housing.")
            avail_housing = "N/A"
            occupied = "N/A"
            vacancy = "N/A"
            homeowner_vacancy = 0
            rental_vacancy = 0

        # Filtered DataFrames remain copies of all data
        filtered_fmr_df = df_fl_fmr.copy()
        filtered_ami_median_df = df_ami_median.copy()
        filtered_ami_housing_df = df_ami_housing.copy()
        filtered_ami_rental_df = df_ami_rental.copy()
        filtered_ami_jobs_df = df_ami_jobs.copy()
        print("Selected 'All Counties'. Showing all data.")
    else:
        # Filter DataFrames based on 'COUNTY_NORM' column
        filtered_fmr_df = df_fl_fmr[df_fl_fmr['COUNTY_NORM'] == county_norm].copy()
        filtered_ami_median_df = df_ami_median[df_ami_median['COUNTY_NORM'] == county_norm].copy()
        filtered_ami_housing_df = df_ami_housing[df_ami_housing['COUNTY_NORM'] == county_norm].copy()
        filtered_ami_rental_df = df_ami_rental[df_ami_rental['COUNTY_NORM'] == county_norm].copy()
        filtered_ami_jobs_df = df_ami_jobs[df_ami_jobs['COUNTY_NORM'] == county_norm].copy()
        
        # Access the selected county's row by filtering
        pop_filtered = df_pop[df_pop['COUNTY_NORM'] == county_norm]
        avail_housing_filtered = df_avail_housing[df_avail_housing['COUNTY_NORM'] == county_norm]

        if not pop_filtered.empty:
            pop_value = pop_filtered['Population: 2022'].values[0]
        else:
            print(f"Error: County '{county_norm}' not found in df_pop.")
            pop_value = "N/A"

        if not avail_housing_filtered.empty:
            avail_housing = avail_housing_filtered['Available Housing Units'].values[0]
            occupied = avail_housing_filtered['Occupied Snapshot'].values[0]
            vacancy = avail_housing_filtered['Vacancy Snapshot'].values[0]
            homeowner_vacancy = avail_housing_filtered['Vacancy Rate: Homeowner'].values[0]
            rental_vacancy = avail_housing_filtered['Vacancy Rate: Rental'].values[0]
        else:
            print(f"Error: County '{county_norm}' not found in df_avail_housing.")
            avail_housing = "N/A"
            occupied = "N/A"
            vacancy = "N/A"
            homeowner_vacancy = 0
            rental_vacancy = 0

        print(f"Filtered data for county: {county_norm}")

    # Update number indicators
    pop_indicator.object = f"### Population: 2022\n\n<span style='font-size:24px; color:#2a9d8f;'>{int(pop_value):,}</span>"
    available_housing_indicator.object = f"### Available Housing Units\n\n<span style='font-size:24px; color:#2a9d8f;'>{int(avail_housing):,}</span>"
    occupied_indicator.object = f"### Occupied Snapshot\n\n<span style='font-size:24px; color:#2a9d8f;'>{int(occupied):,}</span>"
    vacancy_indicator.object = f"### Vacancy Snapshot\n\n<span style='font-size:24px; color:#2a9d8f;'>{int(vacancy):,}</span>"

    # Update gauge values with type checking
    homeowner_vacancy_gauge.value = homeowner_vacancy if isinstance(homeowner_vacancy, (int, float)) else 0
    rental_vacancy_gauge.value = rental_vacancy if isinstance(rental_vacancy, (int, float)) else 0

    # Remove duplicates
    filtered_fmr_df = filtered_fmr_df.drop_duplicates()
    filtered_ami_median_df = filtered_ami_median_df.drop_duplicates()
    filtered_ami_housing_df = filtered_ami_housing_df.drop_duplicates()
    filtered_ami_rental_df = filtered_ami_rental_df.drop_duplicates()
    filtered_ami_jobs_df = filtered_ami_jobs_df.drop_duplicates()    

    # Update the Choropleth map
    if event.new == 'All Counties':
        data_for_map = df_fl_fmr.copy()
        title = 'Florida Counties Map - Population 2022'
    else:
        data_for_map = filtered_fmr_df.copy()
        title = f'Florida County Map - {selected_county} (Population 2022)'

    # Ensure 'fips5' is string type and zero-padded
    data_for_map['fips5'] = data_for_map['fips5'].astype(str).str.zfill(5)

    try:
        fig = px.choropleth(
            data_for_map,
            geojson='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json',
            locations='fips5',
            color='pop2022',  # Use numerical column for color scaling
            scope='usa',
            labels={'COUNTY': 'FL Counties'},
            title=title,
            hover_data=['COUNTY', 'pop2022'],  # Ensure these columns exist
            template='plotly_dark'
        )
    except Exception as e:
        pn.state.notifications.error(f"Error creating Choropleth map: {e}")
        print(f"Error creating Choropleth map: {e}")
        print(f"Data for county '{event.new}' not found.")
        pop_indicator.object = f"### Population: 2022\n\n<span style='font-size:24px; color:#2a9d8f;'>N/A</span>"
        available_housing_indicator.object = f"### Available Housing Units\n\n<span style='font-size:24px; color:#2a9d8f;'>N/A</span>"
        occupied_indicator.object = f"### Occupied Snapshot\n\n<span style='font-size:24px; color:#2a9d8f;'>N/A</span>"
        vacancy_indicator.object = f"### Vacancy Snapshot\n\n<span style='font-size:24px; color:#2a9d8f;'>N/A</span>"
        homeowner_vacancy_gauge.value = 0
        rental_vacancy_gauge.value = 0
        loading_spinner.visible = False
        return

    fig.update_geos(fitbounds="locations", visible=False)

    # Update the Plotly pane
    plotly_pane.object = fig
    print("Choropleth map updated successfully.")

    # Update Tabulator tables
    tabulator_1.value = filtered_fmr_df
    tabulator_2.value = filtered_ami_median_df
    tabulator_3.value = filtered_ami_housing_df
    tabulator_4.value = filtered_ami_rental_df
    tabulator_5.value = filtered_ami_jobs_df
    print("Tabulator tables updated successfully.")

    # Update visibility of Tables and Burden Pane based on selection
    if event.new == 'All Counties':
        tabulator_1.visible = False
        tabulator_2.visible = False
        tabulator_3.visible = False
        tabulator_4.visible = False
        tabulator_5.visible = False
        print("All tables hidden.")
    else:
        tabulator_1.visible = True
        tabulator_2.visible = True
        tabulator_3.visible = True
        tabulator_4.visible = True
        tabulator_5.visible = True
        print("Tables made visible.")

    # Update the bar chart with plot_chart_structure
    structure_plot = plot_chart_structure(event.new)
    structure_plot_pane.object = structure_plot  # Update the Plotly pane with the new plot
    bedrooms_plot = plot_chart_bedrooms(event.new)
    bedrooms_plot_pane.object = bedrooms_plot  # Update the Plotly pane with the new plot
    rooms_plot = plot_chart_rooms(event.new)
    rooms_plot_pane.object = rooms_plot  # Update the Plotly pane with the new plot
    built_plot = plot_chart_built(event.new)
    built_plot_pane.object = built_plot  # Update the Plotly pane with the new plot
    value_plot = plot_chart_value(event.new)
    value_plot_pane.object = value_plot  # Update the Plotly pane with the new plot
    
    loading_spinner.visible = False  # Hide loading spinner
    print("---\n")  # Separator for readability



# ---------------------------
# Attaching the Reactive Function as a Callback
# ---------------------------

# Attaching update_dashboard function to the select_county widget
select_county.param.watch(update_dashboard, 'value')

# ---------------------------
# Dashboard Layout with Tabs
# ---------------------------

pn.config.raw_css.append(custom_css)

# Arrange number indicators in a 2x2 grid
numbers_grid = pn.GridSpec(sizing_mode='stretch_width', max_width=600)

numbers_grid[0, 0] = pop_indicator
numbers_grid[0, 1] = available_housing_indicator
numbers_grid[1, 0] = occupied_indicator
numbers_grid[1, 1] = vacancy_indicator

# Tab 1: Map and County Selector
# Vacancy Rate Table
vacancy_table = pn.pane.HTML("""
<table style="width:300px; border: 1px solid #9F9D9D; border-collapse: collapse; align: left">
  <thead>
    <tr style="background-color: #6200ea">
      <th style="border: 1px solid #000; padding: 8px; color: #ffffff; letter-spacing: 2px;">Market Type</th>
      <th style="border: 1px solid #000; padding: 8px; color: #ffffff; letter-spacing: 2px;">Homeowners</th>
      <th style="border: 1px solid #000; padding: 8px; color: #ffffff; letter-spacing: 2px;">Renters</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border: 1px solid #000; padding: 8px;">Tight</td>
      <td style="border: 1px solid #000; padding: 8px;">&lt; 1%</td>
      <td style="border: 1px solid #000; padding: 8px;">&lt; 5%</td>
    </tr>
    <tr>
      <td style="border: 1px solid #000; padding: 8px;">Balanced</td>
      <td style="border: 1px solid #000; padding: 8px;">1% &lt; 2%</td>
      <td style="border: 1px solid #000; padding: 8px;">5% &lt; 7%</td>
    </tr>
    <tr>
      <td style="border: 1px solid #000; padding: 8px;">Loose</td>
      <td style="border: 1px solid #000; padding: 8px;">&gt; 2%</td>
      <td style="border: 1px solid #000; padding: 8px;">&gt; 7%</td>
    </tr>
  </tbody>
</table>
""", width=300)

# Define the row for gauges
gauges_row = pn.Row(
    homeowner_vacancy_gauge,
    rental_vacancy_gauge,
    
    sizing_mode='stretch_width'
)
# Define a markdown title for the gauges
gauge_title = pn.pane.Markdown("### Vacancy Rate Gauges", sizing_mode='stretch_width')
gauge_title_chart = pn.pane.Markdown("### Vacancy Rate Chart", sizing_mode='stretch_width')

# Combine the gauge title and gauges row in a single column
gauge_column = pn.Column(
    gauge_title,  # Title for the gauges
    gauges_row,   # Gauges placed below the title
    sizing_mode='stretch_width'
)
gauge_chart = pn.Column(
    gauge_title_chart,  # Title for the gauges
    vacancy_table,   # Chart placed below the title
    sizing_mode='stretch_width'
)

# Wrap numbers_grid and gauge_column in a Column to apply a CSS style for minimum width
numbers_grid_wrapped = pn.Column(
    numbers_grid,
    css_classes=['custom-min-width'],
    sizing_mode='stretch_width'
)

gauge_column_wrapped = pn.Column(
    gauge_column,
    css_classes=['custom-min-width'],
    sizing_mode='stretch_width'
)

# Combine numbers grid and gauges row side by side
housing_metrics = pn.Row(
    numbers_grid_wrapped,
    gauge_column_wrapped,
    gauge_chart,
    sizing_mode='stretch_width'  # Ensure the row stretches to fill available space
)

# Add custom CSS to control the min-width globally
pn.extension(raw_css=[
    """
    .custom-min-width {
        min-width: 500px !important;
    }
    """
])



# Landing Page Tab
tab1_content = pn.Column(
    pn.pane.Markdown("## County Map"),
    pn.pane.Markdown("""
        <div style="font-size: 18px;">
        Select a specific County from the list to see downloadable tabular results in the other tabs. 
        More to come.
        </div>
    """),
    pn.Column(
        pn.Row(
            pn.pane.Markdown("### Select County"),
            select_county,
            loading_spinner,
            sizing_mode='stretch_width',
            height=50
        ),
        plotly_pane,
        pn.Row(
            pn.pane.Markdown("""
                <div style="font-size: 18px;">
                Available Housing Metrics
                </div>
            """),
            sizing_mode='stretch_width',
            height=40
        ),
        # Add the housing metrics below the "Available Housing Metrics" markdown
        #housing_metrics
        pn.Row(
            housing_metrics
        ),
        pn.Row(
            pn.pane.HTML("""
<div style="background-color: #ededed; margin:4px 8px; padding: 4px 10px">
<h3>Homeowner Vacancy Rate (HVR)</h3>
<p>The homeowner vacancy rate measures the percentage of vacant homes that are for sale.</p>
<ul>
    <li><strong>Low (tight market)</strong>: HVR below <strong>1%</strong>
        <ul>
            <li>Indicates a housing shortage or very limited availability.</li>
        </ul>
    </li>
    <li><strong>Average (balanced market)</strong>: HVR between <strong>1% and 2%</strong>
        <ul>
            <li>A vacancy rate in this range suggests a balanced market, where supply and demand for homes are relatively even.</li>
        </ul>
    </li>
    <li><strong>Surplus (loose market)</strong>: HVR above <strong>2%</strong>
        <ul>
            <li>A higher vacancy rate indicates surplus availability, which could depress home prices or indicate that homes are remaining on the market longer due to lower demand.</li>
        </ul>
    </li>
</ul>

<h3>Rental Vacancy Rate (RVR)</h3>
<p>The rental vacancy rate measures the percentage of rental units that are vacant and available for rent.</p>
<ul>
    <li><strong>Low (tight market)</strong>: RVR below <strong>5%</strong>
        <ul>
            <li>A rental vacancy rate below 5% suggests a housing shortage in the rental market, potentially leading to higher rents and more competition for available units.</li>
        </ul>
    </li>
    <li><strong>Average (balanced market)</strong>: RVR between <strong>5% and 7%</strong>
        <ul>
            <li>A vacancy rate in this range indicates a balanced market where rental supply generally meets demand.</li>
        </ul>
    </li>
    <li><strong>Surplus (loose market)</strong>: RVR above <strong>7%</strong>
        <ul>
            <li>A rate above 7% points to a surplus in the rental market, which can result in falling rents, more incentives from landlords, and longer vacancies for rental properties.</li>
        </ul>
    </li>
    <li><strong>Note:</strong><br />
        A very high rental vacancy rate in FL can be attributed to several factors. Coastal areas like Gulf County often experience fluctuating housing demand due to seasonal tourism, hurricanes, and economic changes. A significant portion of properties might also be held as second homes or vacation rentals, leading to a higher vacancy rate. Additionally, smaller populations and economic shifts can result in slower absorption of rental units compared to larger urban areas, further contributing to the elevated vacancy rate.
    </li>
</ul>
</div>
""")

        ),
        pn.Row(
            
            update_plot_structure,
            sizing_mode='stretch_width'
        ),
        pn.Row(
            #structure_plot_pane,  # Gauges with title on the right
            update_plot_rooms,
            update_plot_bedrooms,
            sizing_mode="stretch_width"
        ),
        pn.Row(
            #structure_plot_pane,  # Gauges with title on the right
            update_plot_value,
            update_plot_built,
            sizing_mode="stretch_width"
        ),
        pn.Row(
            pn.pane.Markdown("""
                <div style="font-size: 10px;">
                Authored&nbsp; by <a href="https://www.linkedin.com/in/derek-stringfellow/"  target="_blank">Derek Stringfellow, PMP, CSPO, CSM</a>
                </div>
            """),
            sizing_mode='stretch_width',
            height=40
        )
    )
)

tab2_content = pn.Column(
    pn.pane.Markdown("## HUD Fair Market Rent"),
    pn.pane.Markdown("""
        <div style="font-size: 18px;"> 
        More to come.
        </div>
    """),
    pn.Column(
        pn.Row(
            table_1_layout,
            #sizing_mode='stretch_width',
            height=250
        ),
        
        # Add the housing metrics below the "Available Housing Metrics" markdown
        #housing_metrics
        pn.Row(
            table_2_layout
        ) 
    )
)
# Add the new pane as a new tab
tabs = pn.Tabs(
    ("Available Housing", tab1_content),
    ("Fair Market Rents, Median Income", tab2_content),
    #("AMI Median Income", table_2_layout),
    ("Housing Limits by AMI", table_3_layout),
    ("Rental Limits by AMI", table_4_layout),
    ("Wage and Occupation", table_5_layout),
    ("Housing Cost Burden", burden_layout),
    sizing_mode='stretch_both'
)

# ---------------------------
# Display the Dashboard with Tabs
# ---------------------------

# To serve as a standalone application, uncomment the line below:
pn.serve(tabs, threaded=True)


Original FMR DataFrame Columns: ['stusps', 'state', 'hud_area_code', 'COUNTY', 'county_town_name', 'metro', 'hud_area_name', 'fips', 'pop2022', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4', 'DataSource ', 'Report Name', 'URL']
Filtered FL FIPS DataFrame Columns: ['stusps', 'state', 'hud_area_code', 'COUNTY', 'county_town_name', 'metro', 'hud_area_name', 'fips', 'pop2022', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4', 'DataSource', 'Report Name', 'URL', 'fips5']
'2 Bedroom FMR ($)' column successfully renamed and exists.
 homewner_vacancy is, 2.1
 homewner_vacancy type, <class 'numpy.float64'>


<StoppableThread(Thread-11 (get_server), started 66676)>

Launching server at http://localhost:63293
